In [ ]:
# This analysis will allow us to get a quick summary of multiple article about a specific topic.
# We will start by calling a few libraries such as numpy, pandas, and nltk

In [7]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /Users/Youssoufj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
articlesdf = pd.read_csv("tennis_articles_v4.csv")

In [ ]:
# Let's make sure we have the right file, by viewing the head of the data frame.

In [9]:
articlesdf.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [ ]:
# A quick viewing of an article using its index.

In [10]:
articlesdf['article_text'][2]

'Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three days to decide if he would commit to the controversial competition. Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment. "They only left me three days to decide", Federer said. "I didn\'t to have time to consult with all the people I had to consult. "I could not make a decision in that time, so I told them to do what they wanted." The 20-time Grand Slam champion has voiced doubts about the wisdom of the one-week format to be introduced by organisers Kosmos, who have promised the International Tennis Federation up to $3 billion in prize money over the next quarter-century. The competition is set to feature 18 countries in the November 18-24 finals in Madrid next year, and will replace the classic home-

In [8]:
# Tokenization of sentences 

In [11]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in articlesdf['article_text']:
    sentences.append(sent_tokenize(s))
    
sentences = [y for x in sentences for y in x]

In [10]:
# Let's try it

In [12]:
sentences[:2]

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much."]

In [25]:
# Unzip GloVe from Stanford U.
!unzip glove.6B*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [13]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word]= coefs
f.close()

In [14]:
len(word_embeddings)

400000

In [15]:
# Let's remove punctuations, numbers, and other 

clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# Lowering the cases :)

clean_sentences = [s.lower() for s in clean_sentences]

In [16]:
# Let's take care of Too commonly used words like: the...

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Youssoufj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [18]:
# function for stopwords

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [19]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [37]:
# Words vectors

In [20]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()


In [24]:
# Creating vectors for our sentences

sentence_vectors = []
for i in clean_sentences:
    if len(i) !=0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
            v = np.zeros((100,))
            sentence_vectors.append(v)

In [25]:
# Using the cosine similarity approach

sim_mat = np.zeros([len(sentences), len(sentences)])

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
#Initialize the matrix

for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), 
sentence_vectors[j].reshape(1,100))[0,0]

IndexError: list index out of range